<a href="https://colab.research.google.com/github/ucfilho/marquesgabi_paper_fev_2021/blob/main/ANN_%20better_resolution/PSD_histogram_parte_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import zipfile
#import random
from random import randint
from PIL import Image
import re
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
#import scikit-image
import skimage
import pandas as pd
from collections import Counter

In [2]:
!pip install mahotas

In [3]:
import mahotas.features.texture as mht
import mahotas.features

In [4]:
!git clone https://github.com/ucfilho/marquesgabi_fev_2020 #clonar do Github
%cd marquesgabi_fev_2020
import Go2BlackWhite
import Go2Mahotas

fatal: destination path 'marquesgabi_fev_2020' already exists and is not an empty directory.
/content/marquesgabi_fev_2020


In [5]:
!git clone https://github.com/marquesgabi/Doutorado
%cd Doutorado

Transfere='Fotos_Grandes_3cdAmostra.zip'
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()

fatal: destination path 'Doutorado' already exists and is not an empty directory.
/content/marquesgabi_fev_2020/Doutorado


In [6]:
Size=1200 # tamanho da foto
ww,img_name=Go2BlackWhite.BlackWhite(Transfere,Size) #Pegamos a primeira foto Grande
img=ww[0] 
# this is the big image we want to segment 
# ww[0], change it if you want to segment another picture

In [7]:
!git clone https://github.com/ucfilho/MarquesGabi_Routines
%cd MarquesGabi_Routines

fatal: destination path 'MarquesGabi_Routines' already exists and is not an empty directory.
/content/marquesgabi_fev_2020/Doutorado/MarquesGabi_Routines


In [8]:
from segment_filter_not_conclude_parte_03 import Segmenta  # got image provided segmented

In [9]:
n1 = 20 # neurons in hidden layer
n2 = 20 # neurons in hidden layer
c =10 # size to segment
d = 1000  # size to segment

In [10]:
df=Segmenta(img,c,d)
df = df.dropna()

In [11]:
print(df)

    Width           0           1  ...        9997        9998        9999
0     643   42.485558   50.164452  ...  102.605156  110.917328  127.077881
1     572   76.593285   55.672649  ...    2.797252    1.605066    1.283633
2     634   84.843262   88.617203  ...    3.094090    1.625829    1.204112
3     144   97.964508   97.186722  ...    0.000000    0.000000    0.482253
4     798   94.035477   90.899506  ...    3.189384    1.656397    1.017581
5      55   24.000000   24.000000  ...   66.000000   64.363632   64.000000
6     434   85.800606   85.186157  ...   83.376945   81.921188   79.367523
7     911   70.954414   79.081161  ...    3.017180    1.789618    0.890205
8     665   61.562546   61.333031  ...   99.303741   96.018471   94.494484
9     365  108.000008  108.138489  ...    9.841059    9.241696   10.495214
10    801   95.560631   91.469124  ...    3.186560    1.657416    1.014338
11    349  129.297211  129.722397  ...   42.716908   27.425552    5.901511
12    953   61.741478   6

In [12]:
!git clone https://github.com/ucfilho/MarquesGabi_Routines
%cd MarquesGabi_Routines

Cloning into 'MarquesGabi_Routines'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 120 (delta 55), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (120/120), 124.74 MiB | 21.31 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/marquesgabi_fev_2020/Doutorado/MarquesGabi_Routines/MarquesGabi_Routines


In [13]:
# leitura dos dados
df1=pd.read_excel("ImgGrao_a.xlsx")
df2=pd.read_excel("ImgGrao_b.xlsx")
df3=pd.read_excel("ImgIndef.xlsx")
df4=pd.read_excel("ImgVarios.xlsx")
df5=pd.read_excel("ImgBuraco.xlsx")

In [14]:
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
df.head()
df_all = df.copy()

In [15]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(180, 10002)
(179, 10002)
(180, 10002)
(162, 10002)
(180, 10002)


In [16]:
w = df_all['Type']
y = []
for wi in w:
  if(wi == 'G'):
    y.append(1)
  elif(wi == 'B'):
    y.append(2)
  elif(wi == 'I'):
    y.append(2)
  elif(wi == 'V'):
    y.append(2)

In [17]:
df_all.drop(['Unnamed: 0','Type'], axis='columns', inplace=True)

In [18]:
y_ann = y.copy()
X =np.array(df_all.copy())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True, random_state=42)

In [19]:
X_test.shape

(265, 10000)

In [20]:
model = MLPClassifier(hidden_layer_sizes=(n1,n2), activation='tanh',solver='adam',random_state=1, max_iter=300).fit(X_train,y_train)
prediction = model.predict(X_test)
  
y =np.copy(y_test)
data = {'y_true': y_test,'y_predict': prediction}  # este dado esta no formato de dicionario

df = pd.DataFrame(data, columns=['y_true','y_predict'])

confusion_matrix = pd.crosstab(df['y_true'], df['y_predict'], rownames=['Actual'], colnames=['Predict'])
# print(confusion_matrix)

y_true = df['y_true']
y_pred = df['y_predict']
print('n1, n2 =',n1,n2,'true',Counter(y_true),'calc',Counter(y_pred))

n1, n2 = 20 20 true Counter({2: 159, 1: 106}) calc Counter({2: 265})


In [21]:
!git clone https://github.com/ucfilho/marquesgabi_paper_fev_2021
%cd marquesgabi_paper_fev_2021

from Get_PSDArea_New import PSDArea
from histogram_fev_2021 import PSD
from GetBetterSegm import GetBetter

Cloning into 'marquesgabi_paper_fev_2021'...
remote: Enumerating objects: 529, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 529 (delta 178), reused 0 (delta 0), pack-reused 239
Receiving objects: 100% (529/529), 3.39 MiB | 8.28 MiB/s, done.
Resolving deltas: 100% (315/315), done.
/content/marquesgabi_fev_2020/Doutorado/MarquesGabi_Routines/MarquesGabi_Routines/marquesgabi_paper_fev_2021


In [22]:
!git clone https://github.com/marquesgabi/Doutorado
%cd Doutorado

Transfere='Fotos_Grandes_3cdAmostra.zip'
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()

Cloning into 'Doutorado'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 311 (delta 24), reused 0 (delta 0), pack-reused 250
Receiving objects: 100% (311/311), 91.61 MiB | 40.07 MiB/s, done.
Resolving deltas: 100% (147/147), done.
/content/marquesgabi_fev_2020/Doutorado/MarquesGabi_Routines/MarquesGabi_Routines/marquesgabi_paper_fev_2021/Doutorado


In [23]:
Size=1200 # tamanho da foto
ww,img_name=Go2BlackWhite.BlackWhite(Transfere,Size) #Pegamos a primeira foto Grande
img=ww[0] 
# this is the big image we want to segment 
# ww[0], change it if you want to segment another picture

In [24]:
!git clone https://github.com/ucfilho/MarquesGabi_Routines
%cd MarquesGabi_Routines

Cloning into 'MarquesGabi_Routines'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 120 (delta 55), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (120/120), 124.74 MiB | 20.97 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/marquesgabi_fev_2020/Doutorado/MarquesGabi_Routines/MarquesGabi_Routines/marquesgabi_paper_fev_2021/Doutorado/MarquesGabi_Routines


In [25]:
df=Segmenta(img,c,d)
# df = df.dropna()

In [ ]:
cont = 0; num =25
img_graos = []
Width_new = []
img=ww[0] 
global_cont = 0
while( cont < num):
  global_cont = global_cont + 1
  print('global=',global_cont,'cont =', cont)
  df=Segmenta(img,c,d)
  df_ann =df.copy()
  Width = df['Width']
  del df_ann['Width']
  result = np.array(df_ann)/255

  df = df.dropna()
  
  prediction = model.predict(result)
  loc_grao =[];k=0
  for i in prediction:
    # print('i= (1 grao, 2 nao grao',i)
    if( i == 1):
      img_graos.append(df.iloc[k,:])
      Width_new.append(Width.iloc[k])
      cont = cont + 1
    k = k +1
img_graos = pd.DataFrame(img_graos)
print(img_graos)

global= 1 cont = 0
global= 2 cont = 0
global= 3 cont = 0
global= 4 cont = 0
global= 5 cont = 0
global= 6 cont = 0
global= 7 cont = 0


In [ ]:
Width=np.array(Width_new)

In [ ]:
!git clone https://github.com/ucfilho/marquesgabi_out_2020
%cd marquesgabi_out_2020
PSD_imageJ = 'Areas_ImageJ.csv'
PSD_new = pd.read_csv(PSD_imageJ)
print(PSD_new.head(3))

In [ ]:
Area_All, Diameter_All=PSDArea(img_graos) 

In [ ]:
img_graos.head()

In [ ]:
Area = np.array(PSD_new['Area'])
diam_teste = []
for A in Area:
  diam_teste.append((4*A/np.pi)**0.5) 

Diam1 = [ (4*A/np.pi)**0.5 for A in Area]

In [ ]:
 wt1 = np.ones(len(Diam1)) / len(Diam1)*100
 wt2 = np.ones(len(Diameter_All)) / len(Diameter_All)*100
 X = pd.DataFrame([Diam1,Diameter_All])
 wts = pd.DataFrame([wt1,wt2])
plt.hist(X,weights=wts)
plt.legend(['Image J','CNN'])

In [ ]:
counts, bins, bars = plt.hist(X,weights=wts)
print(bars)
print(bins)
print(counts)

In [ ]:
print(counts.sum())

In [ ]:

plt.hist([Diam1,Diameter_All])
plt.legend(['Image J','CNN'])


In [ ]:
[Diam1,Diameter_All]